# These are the plots the in the modern time after ratatouille

In [17]:
import plotly.express as px
import pandas as pd
import numpy as np
from collections import Counter
import ast
from plotly.colors import sample_colorscale
import plotly.graph_objects as go
import plotly.colors

In [18]:
MOVIES = pd.read_csv("cleaned_data.csv")
MOVIES.head()

,wikipedia_movie_ID,name,release_date,revenue,runtime,languages,countries,genres,release_year,vote_average,...,Female actor count,Male actor count,N/A actor count,Actors 0-20,Actors 20-30,Actors 30-40,Actors 40-60,Actors 60+,release_decade,plot_length
0,975900,ghosts of mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2001.0,5.127,...,6,11,0,0,2,3,7,5,2000.0,202
1,3196793,getting away with murder: the jonbenét ramsey ...,2000-02-16,0.0,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",2000.0,0.000,...,4,11,0,1,0,1,6,7,2000.0,34
2,13696889,the gangsters,1913-05-29,0.0,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",1913.0,6.000,...,0,1,0,0,0,0,0,1,1910.0,16
3,10408933,alexander's ragtime band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04t36"": ""Musical"", ""/m/01z4y"": ""Comedy"", ...",1938.0,6.600,...,2,2,0,0,2,2,0,0,1930.0,49
4,6631279,little city,1997-04-04,0.0,93.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ...",1997.0,3.833,...,4,2,0,0,1,4,1,0,1990.0,121


In [19]:
MOVIES.columns

Index(['wikipedia_movie_ID', 'name', 'release_date', 'revenue', 'runtime',
       'languages', 'countries', 'genres', 'release_year', 'vote_average',
       'vote_count', 'adult', 'budget', 'popularity', 'plot',
       'original_genres', 'new_genres', 'genre_hot', 'death', 'love',
       'tragedy', 'sentiment', 'Character Count', 'Female actor count',
       'Male actor count', 'N/A actor count', 'Actors 0-20', 'Actors 20-30',
       'Actors 30-40', 'Actors 40-60', 'Actors 60+', 'release_decade',
       'plot_length'],
      dtype='object')

In [20]:
unique_genre = list()
MOVIES["new_genres"].apply(lambda x: unique_genre.extend(ast.literal_eval(x)))
unique_genre = Counter(unique_genre)
unique_genre = {k: v for k, v in sorted(unique_genre.items(), key=lambda item: item[1], reverse=True)}
unique_genre["Other"] = 0
for idx, entry in enumerate(unique_genre):
    if idx>15 and entry != 'Other':
        unique_genre['Other'] += unique_genre[entry]
        unique_genre[entry] = 0
unique_genre = {x:y for x,y in unique_genre.items() if y!=0}
unique_genre = pd.DataFrame.from_dict(unique_genre, orient='index')
unique_genre = unique_genre.rename(columns={'index':'genre', 0:'count'})
unique_genre = unique_genre


colors = plotly.colors.qualitative.Plotly.copy()
colors.insert(0, 'lightgrey')
print(colors)
fig = px.pie(unique_genre, values='count', names=unique_genre.index, title='Number of movie per genre', color_discrete_sequence=colors)
fig.show()

['lightgrey', '#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']


In [21]:
theme_columns = ["death", "love", "tragedy"]

unique_genre = list()
MOVIES["new_genres"].apply(lambda x: unique_genre.extend(ast.literal_eval(x)))
unique_genre = list(set(unique_genre))
df_themes = MOVIES[theme_columns + ['new_genres']]
dict_themes = {}
for genre in unique_genre:
    df_temp = df_themes[df_themes["new_genres"].apply(lambda x: genre in x)]
    dict_themes[genre] = df_temp[theme_columns].mean().values

df_themes = pd.DataFrame.from_dict(dict_themes, orient="index")
df_themes = df_themes.set_axis(theme_columns, axis=1)

import plotly.express as px
import plotly.io as pio

app = Dash(__name__)

app.layout = html.Div([
    html.H4('Theme words ratio per genre'), #, style={'display': 'block', 'margin': 'auto'}
    dcc.Graph(id="graph"),
    html.P("Genres:"),
    dcc.Dropdown(id='genres',
        options=df_themes.index.values,
        value='Drama', clearable=False, style={
            'width': '300px',
            'height': '40px',
            'font-size': '14px',
            'padding': '5px',
        }),
    html.A(html.Button('Save Plot as HTML', id='save-button', n_clicks=0),
        id="download",),
    dcc.Download(id="download")
    # ),
    # html.P("Values:"),
    # dcc.Dropdown(id='values',
    #     options=['total_bill', 'tip', 'size'],
    #     value='total_bill', clearable=False
    # ),
])


@app.callback(
    Output("graph", "figure"), 
    Input("genres", "value"))
def generate_chart(genres, n_clicks):
    df = df_themes.loc[df_themes.index == genres] # replace with your own data source
    fig = go.Figure(data=[go.Pie(labels=df.columns.str.capitalize().values, values=np.round(df.values[0], 3))])
    fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=plotly.colors.qualitative.Plotly, line=dict(color='#000000', width=2)))
        
    fig.update_layout(
        margin=dict(l=20, r=20, t=20, b=20), width=400, height=600,
        paper_bgcolor="LightSteelBlue",)

@app.callback(
    Output("download", "data"),
    Input("download-button", "n_clicks"),
    prevent_initial_call=True
)
def download_app(n_clicks):
    if n_clicks > 0:
        # Generate HTML representation of the app
        html_content = app.index()  # Get the app's full HTML
        return dict(content=html_content, filename="dash_app.html", type="text/html")
    
    return fig
app.run_server(debug=True, use_reloader=False)

---------------------------------------------------------------------------
DuplicateIdError                          Traceback (most recent call last)
DuplicateIdError: Duplicate component id found in the initial layout: `download`



---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: generate_chart() missing 1 required positional argument: 'n_clicks'



Exception in thread Thread-1150 (<lambda>):
Traceback (most recent call last):
  File "c:\Users\fredr\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\fredr\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\fredr\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\fredr\AppData\Local\Programs\Python\Python312\Lib\site-packages\dash\dash.py", line 1857, in <lambda>
    target=lambda: _watch.watch(
                   ^^^^^^^^^^^^^
  File "c:\Users\fredr\AppData\Local\Programs\Python\Python312\Lib\site-packages\dash\_watch.py", line 35, in watch
    walk()
  File "c:\Users\fredr\AppData\Local\Programs\Python\Python312\Lib\site-packages\dash\_watch.py", line 20, in walk
    info = os.stat(path)
           ^^^^^^^^^^^^^
FileNotFoundError: 

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: generate_chart() missing 1 required positional argument: 'n_clicks'



In [30]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

theme_columns = ["death", "love", "tragedy"]

unique_genre = list()
MOVIES["new_genres"].apply(lambda x: unique_genre.extend(ast.literal_eval(x)))
unique_genre = list(set(unique_genre))
df_themes = MOVIES[theme_columns + ['new_genres']]
dict_themes = {}
for genre in unique_genre:
    df_temp = df_themes[df_themes["new_genres"].apply(lambda x: genre in x)]
    dict_themes[genre] = df_temp[theme_columns].mean().values

df_themes = pd.DataFrame.from_dict(dict_themes, orient="index")
df_themes = df_themes.set_axis(theme_columns, axis=1)

# Create figure
fig = go.Figure()


# Add traces for sin, cos, and tan
for genre in df_themes.index.values:
    df = df_themes.loc[df_themes.index == genre]
    fig.add_trace(go.Pie(labels=df.columns.str.capitalize().values, values=np.round(df.values[0], 3) , name=genre))

# Add dropdown menu to toggle traces
fig.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(label=genre,
                     method='update',
                     args=[{'visible': [False if i !=idx else True for i in range(len(df_themes.index.values))]}])
                for idx, genre in enumerate(df_themes.index.values)
            ],
            direction='down',
        )
    ]
)
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=plotly.colors.qualitative.Plotly, line=dict(color='#000000', width=2)))
        
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",)

fig.show()
# Save as HTML
fig.write_html("saved_plot.html", auto_open=False)
